## Genome Assembly using SPAdes

### Introduction to SPAdes

SPAdes was originally designed as a tool to assemble single-cell genomic sequences that usualy comprise uneven sequence coverage. In order to be able to discern such unenven coverages, the authors developed special algorithms to detect such coverages and to account for such regions. Also, to correct for errors that might be introduced during multiple-displacement amplification (MDA), SPAdes incorporates an "error-correction" step to improve sequence quality in those regions.

Since its intial development in 2012, the assembler has evolved to be able to handle different types of sequencing technologies and datasets. Now, it can handle not only single-cell genomic data but also genomic sequences from isolate genomes (cultures), metagenomes. It can also handle latest "next-gen sequencing" (NGS) and 4th-gen sequencing data such as PacBio, Nanopore, etc. SPAdes is hosted at this website (https://cab.spbu.ru/software/spades/) but you can also visit their github page to see the latest version here (https://github.com/ablab/spades). The github page contains most updated information on how to run SPAdes and what options you can use.

In today's computational exercise, you will use SPAdes to assemble a microbial genome sequenced with Illumina sequencing technology. In order to be able to run SPAdes, you will need to install SPAdes through the Miniconda Python package managing system.

### Installing SPAdes

To install SPAdes on your laptop, you will first type this following command:

In [1]:
conda search spades

Loading channels: done
# Name                       Version           Build  Channel             
spades                         3.6.2               0  bioconda            
spades                         3.7.0               0  bioconda            
spades                         3.8.0               0  bioconda            
spades                         3.8.1               0  bioconda            
spades                         3.9.0               0  bioconda            
spades                         3.9.0          py27_1  bioconda            
spades                         3.9.0          py27_2  bioconda            
spades                         3.9.0          py34_1  bioconda            
spades                         3.9.0          py35_1  bioconda            
spades                         3.9.0          py35_2  bioconda            
spades                         3.9.1               0  bioconda            
spades                         3.9.1      h527b516_1  bioconda            
sp

This brings up all the different versions of SPAdes currently on my `bioconda` channel. The latest version tends to be at the bottom of the page. Then to install the latest version, you would just type `conda install spades`. On my laptop, I have already installed a version of SPAdes. To check what version I have already install, I can type like this below:

In [2]:
conda list | grep spades

spades                    3.15.2               he641558_0    bioconda

Note: you may need to restart the kernel to use updated packages.


So, you can see that I have version 3.15.2 installed. If I want to be explicit about the specific version of SPAdes I want to install, I can type like this:

`conda install spades=3.15.2`

This will install version 3.15.2. If you do not specify a version number, then it will install the latest version by default. Once you have SPAdes installed, you should be able to type this command `spades.py` right away in your terminal.

### Running SPAdes

In [3]:
%%bash
spades.py -h

SPAdes genome assembler v3.15.2

Usage: spades.py [options] -o <output_dir>

Basic options:
  -o <output_dir>             directory to store all the resulting files (required)
  --isolate                   this flag is highly recommended for high-coverage isolate and multi-cell data
  --sc                        this flag is required for MDA (single-cell) data
  --meta                      this flag is required for metagenomic data
  --bio                       this flag is required for biosyntheticSPAdes mode
  --corona                    this flag is required for coronaSPAdes mode
  --rna                       this flag is required for RNA-Seq data
  --plasmid                   runs plasmidSPAdes pipeline for plasmid detection
  --metaviral                 runs metaviralSPAdes pipeline for virus detection
  --metaplasmid               runs metaplasmidSPAdes pipeline for plasmid detection in metagenomic datasets (equivalent for --meta --plasmid)
  --rnaviral                  this flag

I have to type `%%bash` before my command in order to be able to show command line outputs in Jupyter Lab environment. When you are typing `spades.py` in your own terminal, you do not need to type this `%%bash` command. As you can see, you are given a number of options that you can specify when running SPAdes. For example, if you are running a metagenome assembly using SPAdes, then you would add an `--meta` flag to indicate the program that you are running a metagenome assembly.

For today's exercise, you should have first downloaded the raw Illumina sequences from the genome sequencing of a strain of *Salmonella enterica* publicly available on NCBI SRA website. The link to the SRA record is here (https://www.ncbi.nlm.nih.gov/sra/SRX9094324) but the actual accession number you will need to use is **SRR12610971**. This is the accession you should query when typing the `ffq` command to get the ftp link.

You have also done processing of this raw data into a form that is suitable to be used as an input for the SPAdes assembler. This mean running the `bbduk.sh` command (please check last week's example to see how you can process these raw sequences to trim away the Illumina adapter sequences and poor-quality regions). 

### Checking sequence qualities

To check sequence qualities of the two `fastq` files you have downloaded previously, you will run the `fastqc` tool. First, using your terminal, nagivate to the folder where you have downloaded these sequences. For example, if you downloaded the sequences in `/home/yourname/data`, then you would type:

`cd /home/yourname/data`

In my case, I have downloaded them into another folder named `data`.


To check the quality of the downloaded sequences, you would run the FastQC tool, which you have installed previously. To run, navigate into the folder containing the data, then type:

`fastqc *.fastq`

Inspect the quality and statistics reported in this html file to see what the report is telling you. Remember, these are paired fastq files and this means "SRR12610971_1_fastqc.html" is only one of the pair. You will need to inspect the second file "SRR12610971_2_fastqc.html" as well.

### Trimming the sequences

To trim the possible Illumina adapter sequences and poor quality regions, you can use several trimming tools such as Trimmomatic, bbduk, or cutadapt. Here, you will use bbduk, which is part of a suite of tools that come with bbmap tool. If you have installed the tool correctly, you can run bbduk with the following commands:

```bash
bbduk.sh -Xmx12g ktrim=r ordered minlen=50 mink=11 tbo=t rcomp=f k=21 ow=t zl=4 \
        qtrim=rl trimq=20 \
        in1=SRR12610971_1.fastq.gz \
        in2=SRR12610971_2.fastq.gz \
        ref=~/databases/adapters/adapters.fa  \
        out1=SRR12610971_1.trimmed.fastq.gz \
        out2=SRR12610971_2.trimmed.fastq.gz
```

Please make sure your `adapter.fa` file is in the same folder as your downloaded sequences. If not, you need to specify where exactly it is located. In this example, I am referencing a specific location of this adapter as `~/databases/adapters/adapters.fa` as it is not in the same folder as my downloaded sequences. The `~` sign is a shortcut to my home folder on the Mac OS (or a home folder in any Unix-type environment). This usually refers to `/home/username` or `/Users/username` for example. So if I am referencing the adapter file's location as `~/databases/adapters/adapters.fa`, then the actual location is something like `/Users/jsaw/databases/adapters/adapters.fa`. 

The `-Xmx12g` parameter I am providing here is to tell `bbduk.sh` that I want to use 12GB of my computer's memory for this task (you do not need to specify this parameter if you wish and the tool will use the maximum amount of memory available). This parameter may look a bit different from the one shown a few weeks ago.

It will take a few minutes to complete this task. Note that I am using the "\\" to enter the command in multiple lines due to the long list of parameters I need to specify with this tool. You can actually type the whole thing in one line without having to type the "\\" but it is more difficult to see all the options in one screen. There are several parameters that I am entering here to tell the trimming tool what to do exactly with the raw sequences. Detailed options/parameters available with this tool can be seen when you type this:

In [4]:
%%bash
bbduk.sh -h


Written by Brian Bushnell
Last modified January 26, 2021

Description:  Compares reads to the kmers in a reference dataset, optionally 
allowing an edit distance. Splits the reads into two outputs - those that 
match the reference, and those that don't. Can also trim (remove) the matching 
parts of the reads rather than binning the reads.
Please read bbmap/docs/guides/BBDukGuide.txt for more information.

Usage:  bbduk.sh in=<input file> out=<output file> ref=<contaminant files>

Input may be stdin or a fasta or fastq file, compressed or uncompressed.
If you pipe via stdin/stdout, please include the file type; e.g. for gzipped 
fasta input, set in=stdin.fa.gz

Input parameters:
in=<file>           Main input. in=stdin.fq will pipe from stdin.
in2=<file>          Input for 2nd read of pairs in a different file.
ref=<file,file>     Comma-delimited list of reference files.
                    In addition to filenames, you may also use the keywords:
                    adapters, artifacts

As you can see, there are several options and what each of these options does when you select a certain one. In some cases, you can specify only one option but not together with another one.

### Run fastqc on quality-trimmed sequences to compare its quality with untrimmed sequences

Now that you have done trimming the raw sequences, you will then check what the quality looks like. Run fastqc again on trimmed sequences by typing `fastqc *.trimmed.fastq.gz` in the folder where the trimmed files are located. You should inspect the quality reports produced by fastqc to make sure it did what you intended it to do.

You can now use the trimmed fastq files as input to run SPAdes.

### Running the assembly on your laptop or home computer

SPAdes can be run on your home desktop or a laptop computer with a given example dataset provided. It will take a while to complete but it will get done. To run, type the following commands (shown as an example):

```bash
spades.py \
    -t 4 \
    --pe1-1 data/SRR12610971_1.trimmed.fastq.gz \
    --pe1-2 data/SRR12610971_2.trimmed.fastq.gz \
    -k 21,33,55,77 \
    -o spades_assembly
```

This example is one of the simplest examples to run a genome assembly using this particular assembler. Again, this tool also has a number of options/parameters that you can specify, depending on the type of data you have, the size of the data, etc. In this example, `-t 4` means I am using **4 cpus** to run this tool. If you have more cpu cores on your computer, you can specify a larger number. However, with larger datasets (such as metagenomic data), you will require not only more cpus but also larger amount of memory. For this, you will require the use of a high-performance computaional (HPC) clusters that will have larger memory and cpu cores.

The `-k 21,33,55,77` means that I am using 4 different k-mers (read the original SPAdes paper) to run this assembly. You can specify more k-mers but this may cause the assembler to run longer and may not necessarily lead to improved assemblies. This is more of an optimization problem. By design, k-mers need to be odd numbers.

The `-o spades_assembly` parameter tells the program to put output files into a new directory named `spades_assembly`. To see more options, type:

`spades.py -h`

This particular data set run on my iMac with 4 cpus took less than 1 hour to complete. You can try running it on your laptop and see how well it runs or how quickly it gets done. Beware: It may not finish during class. If it did not finish during class, you can stop it and rerun it at home. SPAdes has an option to continue an assembly where it was left off. We can visit how to do this at later time.

Next task for you would be to check the assembly quality after it is done. 

### Running the assembly on a high-performance computing cluster

While the previous example of SPAdes assembly using *Salmonella enterica* sequences you have downloaded from NCBI can be done on your laptop, it may run a lot faster on a high-performance computing (HPC) cluster. We have access to an HPC cluster known as Cerberus. We can try to set up our computing environment in much the same way as we did for our laptops. While we had differences in the way we set up Miniconda or other tools, once you're on Cerberus, you will all be working in the same environment so the tool installation process will be the same for everyone.

Cerberus has the following specifications:

- 28 CPU cores
- 128 GB RAM of memory

Also larger datasets such as metagenomes you have downloaded last week are too big to be assembled on your laptop. They will need to be run on much more powerful computers such as Cerberus.

To make sure you have the right computing environment, please log on to Cerberus using the `ssh` command. An example is shown below:

`ssh jsaw@cerberus.arc.gwu.edu`

Here, I am using my username `jsaw` to log in to the remote computer. The `ssh` command is the one that will enable you to log into a remote computer using "secure-shell" protocol. For you, instead of `jsaw`, you will replace that with your GW user id. The password you type when you first log on and when prompted will be the same as your GW password. However, this will not work if your public ssh key has not been copied over to Cerberus. This is something that an IT administration should have done (if you sent an email with your public key a few weeks ago, it should work). After you log off from Cerberus and you log in the next time, you will not need to type your password; the public key will be used to verify your identity.

Once you are logged onto Cerberus and have seen a splash screen message, you should do the following:

1. Download and install Miniconda on Cerberus. Type the following:
   - `wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh`
   - `chmod +x Miniconda3-latest-Linux-x86_64.sh`
   - `bash Miniconda3-latest-Linux-x86_64.sh`
   - Follow the prompts to complete the installation
2. Set up Bioconda channels.
   ```
   conda config --add channels defaults
   conda config --add channels bioconda
   conda config --add channels conda-forge
   conda config --set channel_priority strict
   ```
3. Install `bbmap` and `spades` using `conda`.

Now you should be ready to run necessary tools to do a genome assembly.

In order to run SPAdes on Cerberus, you will need to redownload the fastq files of the *Salmonella enterica* sequences (SRA accession number SRR12610971) using the `ffq` and `wget` commands as instructed last week. So, the order of things to do once you've installed the quality-trimming and genome assembly tools, you should:

1. Make sure you have the `adapter.fa` file on the remote computer.
2. Download the fastq files again using `ffq` and `wget` commands.
3. Trim the sequences using `bbduk.sh` command.
4. Run assembly using `spades.py` command.

**Note on uploading files to Cerberus:** Now you must be wondering how would you upload the `adapter.fa` file from your laptop to Cerberus, a remote computer. Fortunately, in a Unix/Linux environment, it is easy to transfer files between two or more computers as long as you have credentials to access them. We can use the command `rsync` to do this. An example is shown below:

`rsync -avzP adapter.fa jsaw@cerberus.arc.gwu.edu:~/`

This may seems a bit complicated but don't worry! We can break the parts down one by one. The `rsync` command is the main one that should be invoked to get the ball rolling. The `-avzP` are specific "flags" or parameters that I specified with the `rsync` command to ask the program spit out useful information for me. The `z` in the flag means to compress the file on the fly as it is being transferred to a remote computer. The `jsaw@cerberus.arc.gwu.edu` part is to allow me to log on to the remote computer using my username. The last part, `:~/` is the specify the directory where the file should be copied to. In this case, it gets copied to the base directory under my username.

**Important note on running things on Cerberus:** On a high-performance computing cluster, we normally do not just type and commands we've been typing to run bioinformatic tools but rather submit these commands as `sbatch` jobs. Remember what I talked about "login" nodes vs. "compute" nodes. "login" nodes are just for managing users to log in and out of the system but it does not have a high capacity to do the heavy-lifting usually done by "compute" nodes. Therefore, you need to submit an `sbatch` job when you want to run a SPAdes assembly, for example. 

An example `sbatch` script for a SPAdes assembly looks something like this below:

```bash
#!/bin/bash
#SBATCH -o asm%j.out
#SBATCH -e asm%j.err
#SBATCH -D /home/jsaw/exercises
#SBATCH -J ASM
#SBATCH -N 1
#SBATCH -t 1:00:00
#SBATCH -p defq

module load spades/3.14.1

spades.py \
    -t 16 \
    --pe1-1 ../data/SRR12610971_1.trimmed.fastq.gz \
    --pe1-2 ../data/SRR12610971_2.trimmed.fastq.gz \
    -k 21,33,55,77 \
    -o spades_assembly
```

It is simply a text file you need to create on Cerberus using a command-line text editor such as `vim`, `nano` or `emacs`. You can create a file name as `run_spades.sh` and add those lines to the text file. Then you'd usually submit the job by typing `sbatch run_spades.sh`. We will have more detailed examples and exercises on this for next week's exercises.

### Checking the quality of assemblies

Next thing to do after running these assemblies is to assess how well the assembly was and some other basic metrics related to the assembly. Remember, you started with millions of individual DNA fragments which are only a few hundred bases long and the assembler has magically put together these highly complicated puzzle pieces into a more complete picture, i.e., a draft genome assembly. This assembly is not complete by any means. In fact, most microbial genomes sequenced with only Illumina reads will not give you a complete genome after you run an assembly using SPAdes. 

So you are still missing a lot of pieces and thus the assemblies will yield incomplete but partially assembled DNA fragments that are known as "**contigs**". They range in size from only a few hundred to hundreds of thousands of bases. You wouldn't normally inspect them by using Word processor to count these bases. There are tools specifically designed for you to inspect these metrics, thankfully. One of these tools is known as "**Quast**"" and it happens to be the tool written by the same authors as SPAdes. You will install Quast using `conda`.

```bash
conda search quast
conda install quast
```

Before running Quast on your assemblies, you need to first inspect what SPAdes produced after the assembly. Let's check it out.

In [2]:
%%bash
cd /Volumes/BackupWork/teaching/BISC4234_6234/spades_assembly
ls

K21
K33
K55
K77
assembly_graph.fastg
assembly_graph_with_scaffolds.gfa
before_rr.fasta
contigs.fasta
contigs.paths
corrected
dataset.info
input_dataset.yaml
misc
params.txt
scaffolds.fasta
scaffolds.paths
spades.log
tmp


Now, that's a lot of files and folders. Where do you begin? What is the most important thing for you to inspect? The only files you would need after the assembly are `contigs.fasta` and `scaffolds.fasta` files. `contigs.fasta` contains contigs (incomplete DNA fragments of the genome) of *Salmonella enterica* you just assembled. What about `scaffolds.fasta` file? It is basically similar to `contigs.fasta` file but it has scaffolding information. For example, if the SPAdes assembler has determined that two or more contigs should be physically linked due to pairing information from original reads, then it will put them together but with "N" characters to indicate that there are missing pieces that should be filled.

I normally work with contig files instead of scaffold files for a number of reasons. We will get to that when we start working on genome annotation. For now, let's use Quast to inspect the assembly metrics for the contigs. You need to first be in the folder containing the SPAdes assembly. Then type:

```bash
quast.py contigs.fasta -o quast_contigs
```

This will run Quast on the contigs file and it will produce a folder `quast_contigs` which contains the assembly metrics. Now, let's see what Quast found.

In [4]:
%%bash
cd /Volumes/BackupWork/teaching/BISC4234_6234/spades_assembly
cat quast_contigs/report.txt

All statistics are based on contigs of size >= 500 bp, unless otherwise noted (e.g., "# contigs (>= 0 bp)" and "Total length (>= 0 bp)" include all contigs).

Assembly                    contigs
# contigs (>= 0 bp)         135    
# contigs (>= 1000 bp)      67     
# contigs (>= 5000 bp)      53     
# contigs (>= 10000 bp)     51     
# contigs (>= 25000 bp)     43     
# contigs (>= 50000 bp)     32     
Total length (>= 0 bp)      5097086
Total length (>= 1000 bp)   5075927
Total length (>= 5000 bp)   5039163
Total length (>= 10000 bp)  5024899
Total length (>= 25000 bp)  4900465
Total length (>= 50000 bp)  4481295
# contigs                   77     
Largest contig              282699 
Total length                5082069
GC (%)                      52.08  
N50                         170524 
N75                         94555  
L50                         12     
L75                         24     
# N's per 100 kbp           0.00   


It found 135 contigs in the assembly and the total size of the assembly is about 5 Mbp, which is roughly the genome size of *Salmonella enterica*. The largest contig is more than 280 Kbp long! And G+C% of the geome is about 52%. N50 is another metric that is quite important to assess how well the assembly went. That is the number at which 50% of the assembly size is accounted for by contigs that are longer than 170,524 bases. In this example, 12 contigs can account for 50% of the genome, which is great because fewer contigs mean it's closer to finishing the puzzle to obtain the complete genome. Quast also produces some plots of these metrics in PDF format and an html page containing visualization of the contigs. You can open `icarus.html` file using your web brower to inspect these plots, which are interactive. 

**Note:** You cannot view these html files on a remote computer. Let's say if you ran Quast on Cerberus, you will first need to download them to your laptop first and view the file using your default web browser. The output files that end with `.txt`, however, can be viewed on Cerberus using the `less` command.

### Test trimming parameters and run another SPAdes assembly

A lot of microbial genome assemblies and the qualities are impacted by how well (either aggressively or less aggressively) you trim the original raw sequence reads. Now, go back to the step where you used `bbduk.sh` command to trim these reads. Change one or two parameters to see if you can trim more aggressively (eg: increasing `qtrim` to a higher number, for example) and see if your SPAdes assembly improves or got worse. Make sure to name the assembly differently so that you are not overwriting the previous assembly or the SPAdes assembler won't proceed due to previous assembly folder being present. The parameter you need to change is `-o`.